<a href="https://colab.research.google.com/github/andrewppratt/andrewppratt/blob/main/AnimeGan_Video_(Eng).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<b><font color="black" size="+4">Video to Anime Converter</font></b>

<b><font color="black" size="+2">Based on:</font></b>

**GitHub repository**: [AnimeGAN](https://github.com/TachibanaYoshino/AnimeGAN)

Creator: **[Tachibana Yoshino](https://github.com/TachibanaYoshino).**

<b><font color="black" size="+2">Put it all together:</font></b>

GitHub: [@tg-bomze](https://github.com/tg-bomze),
Telegram: [@bomze](https://t.me/bomze),
Twitter: [@tg_bomze](https://twitter.com/tg_bomze).



```
Next, click on the buttons (where the red arrow indicates) in each block in turn. After clicking, wait until the execution is complete.
```


```
If there are any errors, uncheck the delete_log box, take a screenshot and contact the author of Colab.
```





In [1]:
#@title <b><font color="red" size="+3">←</font><font color="black" size="+3"> Clone the repository and install all requirements</font></b>
!pip install youtube_dl
!pip install ffmpeg
import imageio
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import clear_output
import youtube_dl
import io
import base64
import os
import cv2
from google.colab import files
%matplotlib inline

%tensorflow_version 2.x
!git clone  https://github.com/TachibanaYoshino/AnimeGAN.git
%cd AnimeGAN
!wget https://github.com/TachibanaYoshino/AnimeGAN/releases/download/vgg16%2F19.npy/vgg19.npy
!wget https://github.com/TachibanaYoshino/AnimeGAN/releases/download/Haoyao-style_V1.0/Haoyao-style.zip
!unzip /content/AnimeGAN/Haoyao-style.zip -d pppoint
!rm /content/AnimeGAN/Haoyao-style.zip
!cp /content/AnimeGAN/vgg19.npy /content/AnimeGAN/vgg19_weight
!rm /content/AnimeGAN/vgg19.npy
!mkdir img

delete_log = True #@param {type:"boolean"}
if delete_log == True: clear_output()

In [2]:
#@title <b><font color="red" size="+3">←</font><font color="black" size="+3"> Upload video in mp4</font></b>

#@markdown *Enter a link to the video below (for example, YouTube or Twitter), or leave the source_url field blank (in this case, you will be asked to upload the video from your computer).*

#@markdown *Tests were conducted on videos up to 1 minute long. It is likely that long videos can lead to errors.*
source_url = '' #@param {type:"string"}

if source_url == '':
  uploaded = files.upload()
  for fn in uploaded.keys():
    print('User uploaded file "{name}" with length {length} bytes'.format(
        name=fn, length=len(uploaded[fn])))
  file_name = "downloaded_video." + fn.split(".")[-1]
  !mv -f $fn $file_name

else:
  try:
    ydl_opts = {
        'format': 'bestvideo[ext=mp4]+bestaudio[ext=m4a]/mp4',
        'outtmpl': 'downloaded_video.mp4',
        }
    with youtube_dl.YoutubeDL(ydl_opts) as ydl:
      ydl.download([source_url])
    file_name = 'downloaded_video.mp4'

  except BaseException:
    !wget $source_url
    fn = source_url.split('/')[-1]
    file_name = "downloaded_video." + fn.split(".")[-1]
    !mv -f $fn $file_name

!cp -r downloaded_video.mp4 video.mp4
!rm -rf img
!mkdir img
clear_output()
fps_of_video = int(cv2.VideoCapture(file_name).get(cv2.CAP_PROP_FPS))
frames_of_video = int(cv2.VideoCapture(file_name).get(cv2.CAP_PROP_FRAME_COUNT))

delete_log = True #@param {type:"boolean"}
if delete_log == True: clear_output()
#@markdown *If necessary, convert video to mp4 here: https://convert-video-online.com*

#@markdown *If an error occurs while uploading the file from the computer, run this block again. In addition, it is not recommended to upload video that have spaces or dots in the title.*

In [3]:
#@title <b><font color="red" size="+3">←</font><font color="black" size="+3"> Split the video into frames</font></b>

vidcap = cv2.VideoCapture(file_name)
success,image = vidcap.read()
count = 0
success = True
while success:
  cv2.imwrite("img/frame%09d.jpg" % count, image)
  success,image = vidcap.read()
  count += 1

In [4]:
#@title <b><font color="red" size="+3">←</font><font color="black" size="+3"> Start conversion in anime</font></b>

!python test.py --checkpoint_dir /content/AnimeGAN/pppoint --test_dir /content/AnimeGAN/img --style_name H
delete_log = True #@param {type:"boolean"}
if delete_log == True: clear_output()

In [11]:
#@title <b><font color="red" size="+3">←</font><font color="black" size="+3"> Combining frames to video</font></b>

frames = []
img = os.listdir("/content/AnimeGAN/img")
img.sort()
for i in img:
  frames.append(imageio.imread("/content/AnimeGAN/img/"+i))
frames = np.array(frames)
imageio.mimsave("final.mp4", frames, fps=fps_of_video)
!ffmpeg -i downloaded_video.mp4 -vn -ar 44100 -ac 2 -ab 192K -f mp3 sound.mp3
!ffmpeg -i sound.mp3 -i final.mp4 final_audio.mp4

print('Сборка завершена')
delete_log = True #@param {type:"boolean"}
if delete_log == True: clear_output()

In [12]:
#@title <b><font color="red" size="+3">←</font><font color="black" size="+3"> Download final video</font></b>
files.download("final_audio.mp4")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>